In [87]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import random

Инициализируем модели для получения эмбеддингов

In [88]:
model_tiny = SentenceTransformer('cointegrated/rubert-tiny2')
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

Прочитаем данные и сформируем тексты

In [89]:
df_cv = pd.read_csv("../data/cv_hh.csv", sep=";")
df_vac = pd.read_csv("../data/vacancy_hh.csv", sep=";")

In [90]:
df_cv.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано


In [91]:
df_vac.head()

,id,spec_name,spec_id,name,requirement,responsibility,skills,descr
0,10176972,"Программирование, Разработка",221,Ведущий инженер-программист (Delphi),"Знание основ ООП, Delphi XE и выше, основных ш...",NaN,NaN,"Требования: Знание основ ООП, Delphi XE и выш..."
1,15518045,Игровое ПО,475,Программист графики,Желание изучать и внедрять новые технологии. С...,"Работа с текущим поколением консолей Xbox 1, P...","OpenGL,C++,Графика,Rendering,DirectX,Vulkan",Привет! Наша игровая студия Saber Interactive ...
2,16449260,Продажи,225,Специалист по недвижимости,Грамотная устная и письменная речь. Дисциплина...,Все операции (кроме аренды) на рынке жилой нед...,NaN,За 7 лет работы Мы стали лидерами на рынке нед...
3,16897422,Продажи,225,Специалист по продажам услуг,NaN,Работаешь на выделенной территории. Распростра...,NaN,МТС – одна из крупнейших российских экосистем....
4,17913733,Продажи,225,Специалист по продажам услуг,NaN,Работаешь на выделенной территории. Распростра...,NaN,МТС – одна из крупнейших российских экосистем....


In [92]:
df_vac.rename(columns={'spec_name': 'Сфера деятельности', 
                       'name': 'Должность',
                       "responsibility": "Обязанности",
                       "skills": "Навыки",
                       "descr": "Описание вакансии",
                       "requirement": "Требования"}, inplace=True)

In [93]:
df_vac.fillna("Не указано", inplace=True)
df_cv.fillna("Не указано", inplace=True)

Возьмем случайные cv и вакансию и создадим из них текст

In [95]:
cv = [str(col) + ":\n" + "".join(str(df_cv.iloc[3][col]) + "\n") for col in df_cv.iloc[3].index]
cv = "".join(cv)
print(cv)

Пол, возраст:
Мужчина ,  38 лет , родился 25 июня 1980
ЗП:
100000 руб.
Ищет работу на должность::
Веб-разработчик (HTML / CSS / JS / PHP / базы данных; фреймворки, дизайн, интерфейсы, CMS)
Город, переезд, командировки:
Саратов , не готов к переезду , готов к редким командировкам
Занятость:
частичная занятость, проектная работа, полная занятость
График:
гибкий график, удаленная работа
Опыт работы:
Опыт работы 18 лет 9 месяцев  Август 2017 — Апрель  2019 1 год 9 месяцев OpenSoft Саратов , www.opensoftdev.ru Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Инженер-программист Разработка и поддержка сайтов на корпоративной CMS и на WordPress; разработка корпоративной CMS; дизайн, вёрстка, улучшение интерфейсов существующих сайтов, основанных на разных фреймворках и CMS; консультации и техническая поддержка.  Октябрь 2006 — Июнь  2018 11 лет 9 месяцев СГУ им. Н.Г. Чернышевского Сар

In [96]:
vac = [str(col) + ":\n" + "".join(str(df_vac.iloc[2][col]) + "\n") for col in df_vac.iloc[2].index if col not in ["id", "spec_id"]]
vac = "".join(vac)
print(vac)

Сфера деятельности:
Продажи
Должность:
Специалист по недвижимости
Требования:
Грамотная устная и письменная речь. Дисциплина и аккуратность. Стремление к обучению. Знание географии Уфы. Общие знания о процессе продаж, начальные...
Обязанности:
Все операции (кроме аренды) на рынке жилой недвижимости Уфы. Поиск и презентация недвижимости для клиентов. Презентация услуг компании, организация просмотров...
Навыки:
Не указано
Описание вакансии:
За 7 лет работы Мы стали лидерами на рынке недвижимости в РФ и продолжаем расширять горизонты. На данный момент открыты офисы уже в 90городах России. Для оператора недвижимости «ПЕРСПЕКТИВА 24» это-возможность сделать сотни и тысячи людей на шаг ближе к своей мечте. За это время мы зарекомендовали себя как надежное и современное агентство недвижимости. Каждый день мы улучшаем сервис, получая обратную связь от клиентов и партнеров. Должностные обязанности:  Все операции (кроме аренды) на рынке жилой недвижимости Уфы. Поиск и презентация недвижимости д

Рассмотрим значение косинусного сходства для рандомных резюме и вакансии с использованием двух разных энкодеров

In [97]:
cosine_similarity(model.encode([cv, vac]))

array([[0.9999999 , 0.42617628],
       [0.42617628, 0.99999976]], dtype=float32)

In [98]:
cosine_similarity(model_tiny.encode([cv, vac]))

array([[1.       , 0.7621541],
       [0.7621541, 0.9999999]], dtype=float32)

In [99]:
cosine_similarity(model.encode(["Веб-разработчик", "Специалист по недвижимости"]))

array([[1.0000001 , 0.29971707],
       [0.29971707, 1.        ]], dtype=float32)

In [100]:
cosine_similarity(model_tiny.encode(["Веб-разработчик", "Специалист по недвижимости"]))

array([[1.       , 0.7087561],
       [0.7087561, 1.0000002]], dtype=float32)

Модель **paraphrase-multilingual-mpnet-base-v2** дает более точные эмбеддинги, которые позволят нам на основании косинусного сходства более точно подбирать пару резюме+вакансия

Теперь попробуем посчитать эмбеддинги для всего датасета и найти рекомендованные вакансии для рандомного резюме

Для этого сначала создадим текстовое поле для резюме/вакансии и заполним его

In [104]:
cvs, vacs = [], []

for index, row in df_cv.iterrows():
    cv = [str(col) + ":\n" + "".join(str(row[col]) + "\n") for col in row.index if col not in ["Обновление резюме"]]
    cv = "".join(cv)
    cvs.append(cv)

for index, row in df_vac.iterrows():
    vac = [str(col) + ":\n" + "".join(str(row[col]) + "\n") for col in row.index if col not in ["id", "spec_id"]]
    vac = "".join(vac)
    vacs.append(vac)

df_cv["cv_text"] = cvs
df_vac["vac_text"] = vacs

In [106]:
print(df_cv.iloc[0]["cv_text"])

Пол, возраст:
Мужчина ,  39 лет , родился 27 ноября 1979
ЗП:
29000 руб.
Ищет работу на должность::
Системный администратор
Город, переезд, командировки:
Советск (Калининградская область) , не готов к переезду , не готов к командировкам
Занятость:
частичная занятость, проектная работа, полная занятость
График:
гибкий график, полный день, сменный график, вахтовый метод, удаленная работа
Опыт работы:
Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники
Последнее/нынешнее место работы:
МАОУ "СОШ № 1 г.Немана"
Последняя/нынешняя должность:
Системный администратор
Образование и ВУЗ:
Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыс

In [107]:
print(df_vac.iloc[0]["vac_text"])

Сфера деятельности:
Программирование, Разработка
Должность:
Ведущий инженер-программист (Delphi)
Требования:
Знание основ ООП, Delphi XE и выше, основных шаблонов построения пользовательского интерфейса,SQL (Oraclе, MS SQL), умение пользоваться средами разработки. 
Обязанности:
Не указано
Навыки:
Не указано
Описание вакансии:
Требования:  Знание основ ООП, Delphi XE и выше, основных шаблонов построения пользовательского интерфейса,SQL (Oraclе, MS SQL), умение пользоваться средами разработки. Приветствуется знание систем управления версиями (напр. Subversion). Выезд в командировки. Высшее образование профильное Опыт работы от 3 х лет  Условия:  Оформление согласно ТК РФ Достойная заработная плата: оклад + премия 



In [108]:
df_cv.to_csv("../data/datasets_with_text/cv.csv")
df_vac.to_csv("../data/datasets_with_text/vac.csv")

Посчитаем и сохраним эмбеддинги

In [109]:
df_cv["tiny_embedding"] = df_cv["cv_text"].apply(lambda x: model_tiny.encode(x))
df_cv["embedding"] = df_cv["cv_text"].apply(lambda x: model.encode(x))

df_vac["tiny_embedding"] = df_vac["vac_text"].apply(lambda x: model_tiny.encode(x))
df_vac["embedding"] = df_vac["vac_text"].apply(lambda x: model.encode(x))

In [110]:
df_vac.head()

,id,Сфера деятельности,spec_id,Должность,Требования,Обязанности,Навыки,Описание вакансии,vac_text,tiny_embedding,embedding
0,10176972,"Программирование, Разработка",221,Ведущий инженер-программист (Delphi),"Знание основ ООП, Delphi XE и выше, основных ш...",Не указано,Не указано,"Требования: Знание основ ООП, Delphi XE и выш...","Сфера деятельности:\nПрограммирование, Разрабо...","[0.059508625, 0.011022791, -0.03378131, -0.102...","[-0.045974743, 0.09887265, -0.0105135925, -0.0..."
1,15518045,Игровое ПО,475,Программист графики,Желание изучать и внедрять новые технологии. С...,"Работа с текущим поколением консолей Xbox 1, P...","OpenGL,C++,Графика,Rendering,DirectX,Vulkan",Привет! Наша игровая студия Saber Interactive ...,Сфера деятельности:\nИгровое ПО\nДолжность:\nП...,"[0.08854647, -0.0057156086, -0.012761924, -0.0...","[-0.041332595, 0.1011829, -0.0051545943, 0.024..."
2,16449260,Продажи,225,Специалист по недвижимости,Грамотная устная и письменная речь. Дисциплина...,Все операции (кроме аренды) на рынке жилой нед...,Не указано,За 7 лет работы Мы стали лидерами на рынке нед...,Сфера деятельности:\nПродажи\nДолжность:\nСпец...,"[0.09230125, 0.0028123322, 0.014232679, -0.075...","[0.005738664, 0.098482594, -0.013911629, -0.10..."
3,16897422,Продажи,225,Специалист по продажам услуг,Не указано,Работаешь на выделенной территории. Распростра...,Не указано,МТС – одна из крупнейших российских экосистем....,Сфера деятельности:\nПродажи\nДолжность:\nСпец...,"[0.11931744, 0.012647032, -0.011578139, -0.102...","[0.13866234, -0.09461806, -0.013993509, -0.105..."
4,17913733,Продажи,225,Специалист по продажам услуг,Не указано,Работаешь на выделенной территории. Распростра...,Не указано,МТС – одна из крупнейших российских экосистем....,Сфера деятельности:\nПродажи\nДолжность:\nСпец...,"[0.11931744, 0.012647032, -0.011578139, -0.102...","[0.13866234, -0.09461806, -0.013993509, -0.105..."


In [111]:
df_vac.to_csv("../data/datasets_with_embeddings/vac.csv")
df_cv.to_csv("../data/datasets_with_embeddings/cv.csv")

Выберем рандомно резюме и вакансию и попробюуем подобрать для них вакансию/резюме соответственно

In [240]:
cv_num = random.randint(0, len(df_cv) - 1)
vac_num = random.randint(0, len(df_vac) - 1)

In [243]:
sims = {}
tiny_sims = {}

for index, row in df_vac.iterrows():
    sims[index] = cosine_similarity([df_cv.iloc[cv_num]["embedding"], row["embedding"]])[0][1]
    tiny_sims[index] = cosine_similarity([df_cv.iloc[cv_num]["tiny_embedding"], row["tiny_embedding"]])[0][1]


In [244]:
sims = {k: v for k, v in sorted(sims.items(), key=lambda item: item[1], reverse=True)}
tiny_sims = {k: v for k, v in sorted(tiny_sims.items(), key=lambda item: item[1], reverse=True)}

In [246]:
print(df_cv.iloc[cv_num]["cv_text"])

Пол, возраст:
Женщина ,  27 лет , родилась 21 июля 1991
ЗП:
40000 руб.
Ищет работу на должность::
Менеджер интернет-магазина
Город, переезд, командировки:
Москва ,  м. Бабушкинская , не готова к переезду , готова к редким командировкам
Занятость:
полная занятость
График:
полный день
Опыт работы:
Опыт работы 6 лет 3 месяца  Менеджер интернет-магазина 40 000 руб. Информационные технологии, интернет, телеком Продажи Электронная коммерция Занятость: полная занятость График работы: полный день Опыт работы 6 лет 3 месяца Сентябрь 2018 — Апрель  2019 8 месяцев ООО"Профи-Центр" Услуги для бизнеса ... Кадровые агентства Менеджер по персоналу -Поиск, -Холодные звонки, -Размещение вакансий. -Взаимодействие с заведующими магазинов по вопросам прохождения стажировки, результатов испытательного срока. -Осуществление ротации персонала - Ведение ежедневной отчетности Октябрь 2016 — Июль  2018 1 год 10 месяцев "Унция" Москва , www.chay.info/ Продавец-консультант Выявление потребностей, оформление и сбо

In [247]:
for i, vac in enumerate(df_vac.iloc[list(sims.keys())[0:5]]["vac_text"]):
    print(f"Топ {i + 1} вакансия:\n", vac)

Топ 1 вакансия:
 Сфера деятельности:
Инженер
Должность:
Оператор ЧПУ
Требования:
опыт работы.
Обязанности:
Не указано
Навыки:
Пользователь ПК
Описание вакансии:
Вахтовым методом в г. Ростов-на-Дону на комбайновый завод требуются операторы чпу с опытом работыУсловия:-официальное трудоустройство-иногородним предоставляется жилье-спецодежда и медкомиссия за счет компании-зарплата без задержек 2 раза в месяцТребования:-опыт работы

Топ 2 вакансия:
 Сфера деятельности:
Телекоммуникации
Должность:
Электромонтер-релейщик
Требования:
Образование не ниже среднего профессионального. Приветствуется наличие: - III гр. до 1000В по электробезопасности. - отметки «Ростехнадзор» о допуске к специальным...
Обязанности:
Обслуживание, ремонт, восстановление при неисправности системы автоматического перевода трамвайных стрелок по г. Москве. Работа под напряжением 600В на изолированной...
Навыки:
Не указано
Описание вакансии:
Обязанности:  обслуживание, ремонт, восстановление при неисправности системы авто

In [248]:
for i, vac in enumerate(df_vac.iloc[list(tiny_sims.keys())[0:5]]["vac_text"]):
    print(f"Топ {i + 1} вакансия:\n", vac)

Топ 1 вакансия:
 Сфера деятельности:
Оптимизация сайта (SEO)
Должность:
Web-аналитик
Требования:
Знание систем web/app-аналитики: Google Analytics, Google Tag Manager, Яндекс.Метрика, Appsflyer, Firebase, Facebook Business Manager. Базовые знания принципов маркетинга...
Обязанности:
Регулярный анализ результатов рекламных кампаний. Настройка web/app-аналитики и/или подготовка ТЗ для их настройки разработчикам и подрядчикам. 
Навыки:
Google Analytics,Яндекс.Метрика,Google AdWords,Яндекс.Директ,Интернет-реклама
Описание вакансии:
КОМПАНИЯ «ВЕС-МЕДИА» ПРИГЛАШАЕТ КАНДИДАТОВ НА ВАКАНСИЮ «WEB-АНАЛИТИК» НА ТЕЛЕКАНАЛ «ЮВЕЛИРОЧКА»! В ЗОНУ ОТВЕТСТВЕННОСТИ БУДЕТ ВХОДИТЬ:  Регулярный анализ результатов рекламных кампаний Настройка web/app-аналитики и/или подготовка ТЗ для их настройки разработчикам и подрядчикам Помощь в настройке автоматизации регулярных отчетов Слежение за эффективностью на всех этапах воронки продаж сайта и мобильного приложения Отслеживание и анализ результативности акций Анал

Как оказалось модель **paraphrase-multilingual-mpnet-base-v2** дает ужасные результаты, для поиска вакансий по резюме стоит оставить **rubert-tiny**
Теперь посмотрим поиск резюме по вакансии

In [249]:
sims = {}
tiny_sims = {}

for index, row in df_cv.iterrows():
    sims[index] = cosine_similarity([df_vac.iloc[vac_num]["embedding"], row["embedding"]])[0][1]
    tiny_sims[index] = cosine_similarity([df_vac.iloc[vac_num]["tiny_embedding"], row["tiny_embedding"]])[0][1]

In [250]:
sims = {k: v for k, v in sorted(sims.items(), key=lambda item: item[1], reverse=True)}
tiny_sims = {k: v for k, v in sorted(tiny_sims.items(), key=lambda item: item[1], reverse=True)}

In [252]:
print(df_vac.iloc[vac_num]["vac_text"])

Сфера деятельности:
Программирование, Разработка
Должность:
Python developer
Требования:
Отличные знания Python. Уверенное знание Django, Django Rest Framework. Опыт работы с очередями (Celery, RabbitMQ). Опыт работы с базами данных...
Обязанности:
Разработка и поддержка Backend приложения.
Навыки:
Не указано
Описание вакансии:
Технократия – это не просто название нашей компании, но и наша идеология и философия в рамках которой мы работаем и развиваемся. Наша команда состоит из талантливых и амбициозных профессионалов. Вместе мы каждый день доказываем, что являемся одной из лучших команд разработки в России. Чем предстоит заниматься:  Разработка и поддержка Backend приложения.  Мы ожидаем от Ваc:  Отличные знания Python Уверенное знание Django, Django Rest Framework Опыт работы с очередями (Celery, RabbitMQ) Опыт работы с базами данных (Redis, Postgresql) Знание и умение применять паттерны проектирования Опыт построения асинхронных-приложений Опыт проектирования REST API Понимание прин

In [254]:
for i, cv in enumerate(df_cv.iloc[list(sims.keys())[0:5]]["cv_text"]):
    print(f"Топ {i + 1} резюме:\n", cv)

Топ 1 резюме:
 Пол, возраст:
Мужчина ,  24 года , родился 16 июня 1994
ЗП:
100000 руб.
Ищет работу на должность::
Программист Python, Data Scientist, Django разработчик
Город, переезд, командировки:
Томск , не готов к переезду , не готов к командировкам
Занятость:
полная занятость
График:
полный день
Опыт работы:
Опыт работы 2 года 6 месяцев  Программист Python, Data Scientist, Django разработчик 100 000 руб. Информационные технологии, интернет, телеком Web инженер Интернет Web мастер Занятость: полная занятость График работы: полный день Опыт работы 2 года 6 месяцев Ноябрь 2016 — по настоящее время 2 года 6 месяцев ЛИСМО НУ ИИ Программист - Поддерживать программы предшественника ( java, c#) - Разрабатывать приложения для Moodle (лабораторные работы, php + sql + js + html + css + bootstrap) - Разрабатывать инструменты для лаборатории (в частности, разработал технологию автоматизированного создания тестовых заданий на основе применения нейронных сетей, python + keras) Ключевые навыки Те

In [255]:
for i, cv in enumerate(df_cv.iloc[list(tiny_sims.keys())[0:5]]["cv_text"]):
    print(f"Топ {i + 1} резюме:\n", cv)

Топ 1 резюме:
 Пол, возраст:
Мужчина ,  28 лет , родился 11 августа 1990
ЗП:
50000 руб.
Ищет работу на должность::
Специалист по IT
Город, переезд, командировки:
Рязань , готов к переезду (Москва) , не готов к командировкам
Занятость:
частичная занятость, полная занятость
График:
гибкий график, полный день, сменный график
Опыт работы:
Опыт работы 6 лет 9 месяцев  Сентябрь 2012 — по настоящее время 6 лет 9 месяцев ИК Сибинтек Рязань Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Ведущий специалист Оказание технической и IT поддержки группе компаний ТНК-BP/НК Роснефть и ДО. HelpDesk. Оказание технической и информационной поддержки пользовательского ПО. Анализ поступающих обращений. Администрирование прав пользователей по доступу к инф.ресурсам (типовые инструкции). Удаленное решение типовых проблем пользователя с MS OFFICE, PI System, Lims, восстановление работоспособности орг

Здесь обе модели справились неплохо, но заметно, что **paraphrase-multilingual-mpnet-base-v2** дает более точные рекомендации